In [ ]:
!pip install wget
!pip install transformers datasets evaluate tensorflow

In [ ]:
import wget
# import pandas as pd
# from imblearn.over_sampling import RandomOverSampler
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing
# from transformers import TFBertModel
# import tensorflow as tf
# import random
# from transformers import BertTokenizerFast
# from sklearn.metrics import classification_report, confusion_matrix
from transformers import create_optimizer, TFAutoModelForQuestionAnswering, DefaultDataCollator
from datasets import Dataset
import tensorflow as tf
from transformers import AutoTokenizer

In [ ]:
data = wget.download("https://storage.yandexcloud.net/academy.ai/LLM/dialogs.txt")

In [ ]:
with open(data, "r") as file:
  data = file.readlines()

In [ ]:
def get_text(data):
  quest_texts = []
  answer_texts = []
  context_texts = []
  start_index_list = []
  end_index_list = []

  for text in data:
    quest, answer = text.lower().replace(". ", '.').replace('\n', '').split('\t')

    context = f"{quest} {answer}"
    start_index = context.index(answer)
    end_index = start_index + len(answer) - 1

    quest_texts.append(quest)
    answer_texts.append(answer)
    context_texts.append(context)
    start_index_list.append(start_index)
    end_index_list.append(end_index)

  return quest_texts, answer_texts, context_texts, start_index_list, end_index_list

In [ ]:
quest_texts, answer_texts, context_texts, start_index_list, end_index_list  = get_text(data)
print(f"Всего реплик: {len(quest_texts)}")
print(f"quest_texts: {quest_texts[0]}")
print(f"answer_texts: {answer_texts[0]}")
print(f"context_texts: {context_texts[0]}")
print(f"start_index_list: {start_index_list[0]}")
print(f"end_index_list: {end_index_list[0]}")

Всего реплик: 3725
quest_texts: hi, how are you doing?
answer_texts: i'm fine.how about yourself?
context_texts: hi, how are you doing? i'm fine.how about yourself?
start_index_list: 23
end_index_list: 50


In [ ]:
test_len = lambda x: int(x - x*0.1)

train_contexts = context_texts[:test_len(len(context_texts))]
valid_contexts = context_texts[test_len(len(context_texts)):]
print(f"len(train): {len(train_contexts)}, len(valid): {len(valid_contexts)}")

train_questions = quest_texts[:test_len(len(quest_texts))]
valid_questions = quest_texts[test_len(len(quest_texts)):]
print(f"len(train): {len(train_questions)}, len(valid): {len(valid_questions)}")

train_start = start_index_list[:test_len(len(start_index_list))]
valid_start = start_index_list[test_len(len(start_index_list)):]

train_answers = answer_texts[:test_len(len(answer_texts))]
valid_answers = answer_texts[test_len(len(answer_texts)):]

train_end = end_index_list[:test_len(len(end_index_list))]
valid_end = end_index_list[test_len(len(end_index_list)):]

len(train): 3352, len(valid): 373
len(train): 3352, len(valid): 373


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.DataFrame({
    'question' : train_questions,
    'context' : train_contexts,
    'answer' : train_answers,
    'answer_start' : train_start,
    'answer_end' : train_end
})
train_df

,question,context,answer,answer_start,answer_end
0,"hi, how are you doing?","hi, how are you doing? i'm fine.how about your...",i'm fine.how about yourself?,23,50
1,i'm fine.how about yourself?,i'm fine.how about yourself? i'm pretty good.t...,i'm pretty good.thanks for asking.,29,62
2,i'm pretty good.thanks for asking.,i'm pretty good.thanks for asking. no problem....,no problem.so how have you been?,35,66
3,no problem.so how have you been?,no problem.so how have you been? i've been gre...,i've been great.what about you?,33,63
4,i've been great.what about you?,i've been great.what about you? i've been good...,i've been good.i'm in school right now.,32,70
...,...,...,...,...,...
3347,so why are we selling our house?,so why are we selling our house? they're build...,they're building a 3-story apartment building ...,33,92
3348,they're building a 3-story apartment building ...,they're building a 3-story apartment building ...,so we've got to sell before property values go...,61,112
3349,so we've got to sell before property values go...,so we've got to sell before property values go...,yes.i still can't believe our city council all...,53,117
3350,yes.i still can't believe our city council all...,yes.i still can't believe our city council all...,they're probably getting something under the t...,66,116


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

questions = [q.strip() for q in train_df["question"]]
context = [q.strip() for q in train_df["context"]]
answers = [q.strip() for q in train_df["answer"]]
start = train_df["answer_start"].tolist()
end = train_df["answer_end"].tolist()

inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []

for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = start[i]
    end_char = end[i]-1
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

train_df["answer_start"] = start_positions
train_df["answer_end"] = end_positions

import pandas as pd
from datasets import Dataset
data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_train.csv',index=False)
train = Dataset.from_pandas(df)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
test_df = pd.DataFrame({
    'question' : valid_questions,
    'context' : valid_contexts,
    'answer' : valid_answers,
    'answer_start' : valid_start,
    'answer_end' : valid_end
})
test_df

,question,context,answer,answer_start,answer_end
0,"yes, it was.","yes, it was. they said about 30 houses burned ...",they said about 30 houses burned to the ground.,13,59
1,they said about 30 houses burned to the ground.,they said about 30 houses burned to the ground...,and they were expensive houses.,48,78
2,and they were expensive houses.,and they were expensive houses. i feel so sorr...,i feel so sorry for those people.,32,64
3,i feel so sorry for those people.,i feel so sorry for those people. why feel sor...,why feel sorry for rich people?,34,64
4,why feel sorry for rich people?,why feel sorry for rich people? i feel sorry f...,i feel sorry for anyone who loses their home.,32,76
...,...,...,...,...,...
368,that's a good question.maybe it's not old age.,that's a good question.maybe it's not old age....,are you right-handed?,47,67
369,are you right-handed?,are you right-handed? yes.all my life.,yes.all my life.,22,37
370,yes.all my life.,yes.all my life. you're wearing out your right...,you're wearing out your right hand.stop using ...,17,73
371,you're wearing out your right hand.stop using ...,you're wearing out your right hand.stop using ...,but i do all my writing with my right hand.,58,100


In [ ]:
questions = [q.strip() for q in test_df["question"]]
context = [q.strip() for q in test_df["context"]]
answers = [q.strip() for q in test_df["answer"]]
start = test_df["answer_start"].tolist()
end = test_df["answer_end"].tolist()

inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []

for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = start[i]
    end_char = end[i]-1
    sequence_ids = inputs.sequence_ids(i)

    # Поиск начала и конец контекста
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # Если ответ не полностью соответствует контексту, отмечаю его (0, 0).
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # В противном случае это начальная и конечная позиции токена.
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

test_df["answer_start"] = start_positions
test_df["answer_end"] = end_positions

import pandas as pd
from datasets import Dataset
data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_test.csv',index=False)
test = Dataset.from_pandas(df)

In [ ]:
batch_size = 16
num_epochs = 2
total_train_steps = (len(train) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=2)

Epoch 1/2
209/209 [==============================] - 174s 699ms/step - loss: 0.8590 - val_loss: 0.0200
Epoch 2/2
209/209 [==============================] - 141s 676ms/step - loss: 0.0400 - val_loss: 0.0136


In [ ]:
from transformers import pipeline
lambda_rand = lambda x: int(x * 13)

for i in range(5):
  question = quest_texts[lambda_rand(i)]
  context = context_texts[lambda_rand(i)]
  answer = answer_texts[lambda_rand(i)]

  question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
  print(question_answerer(question=question, context=context))
  print(f'Вопрос: {question}')
  print(f'Правильный ответ: {answer}\n\n')

{'score': 0.9977220892906189, 'start': 23, 'end': 50, 'answer': "i'm fine.how about yourself"}
Вопрос: hi, how are you doing?
Правильный ответ: i'm fine.how about yourself?


{'score': 0.9980999231338501, 'start': 22, 'end': 49, 'answer': 'so how have you been lately'}
Вопрос: never better, thanks.
Правильный ответ: so how have you been lately?


{'score': 0.9975985884666443, 'start': 42, 'end': 67, 'answer': 'i started school recently'}
Вопрос: i haven't been better.how about yourself?
Правильный ответ: i started school recently.


{'score': 0.995826780796051, 'start': 42, 'end': 74, 'answer': "me too.i can't wait until winter"}
Вопрос: i really wish it wasn't so hot every day.
Правильный ответ: me too.i can't wait until winter.


{'score': 0.9954032301902771, 'start': 55, 'end': 108, 'answer': "i know what you mean, but i'd rather be cold than hot"}
Вопрос: i enjoy the winter, but it gets really cold sometimes.
Правильный ответ: i know what you mean, but i'd rather be cold than hot.
